In [39]:
import rosbag
from sensor_msgs.msg import Imu, NavSatFix
import scipy.io as sio
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import interp1d
import numpy as np

### load data

In [2]:
bag = rosbag.Bag('/home/crange/dev/calib_ws/calib_data/car.bag')
ins = []
imu = []
for topic, msg, t in bag.read_messages():
    if topic == "/novatel/oem7/inspva":
        ins.append(msg)
    if topic == "/imu/data":
        imu.append(msg)

In [48]:
e = []
for i in range(len(ins)):
    e.append([ins[i].roll, ins[i].pitch, ins[i].azimuth])

euler = np.array(e)

In [51]:
x = np.linspace(0, 1, len(euler))
f = interp1d(x, euler, axis=0)

new_x = np.linspace(0, 1, 5384)
new_euler = f(new_x)

In [53]:
len(new_euler)

5384

In [15]:
output_data = []
for i in range(len(ins)):
    t = ins[i].header.stamp.secs + ins[i].header.stamp.nsecs * 1e-9
    r = R.from_euler('xyz', [ins[i].roll, ins[i].pitch, ins[i].azimuth])
    q = r.as_quat()  # xyzw
    output_data.append([round(t, 9), 0.0, 0.0, 0.0, round(q[0], 9), round(q[1], 9), round(q[2], 9), round(q[3], 9)])

In [19]:
save_path = 'output/odometry.txt'

with open(save_path, "w", encoding="utf-8") as file1:
    for i in range(len(ins)):
        print("{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}".
              format(output_data[i][0], 9, 
                     output_data[i][1], 9,
                     output_data[i][2], 9,
                     output_data[i][3], 9,
                     output_data[i][4], 9,
                     output_data[i][5], 9,
                     output_data[i][6], 9,
                     output_data[i][7], 9), file=file1)
        # print(output_data[i][0], output_data[i][1], output_data[i][2], output_data[i][3], output_data[i][4], output_data[i][5], output_data[i][6], output_data[i][7], file=file1)

### imu output

In [34]:
temp1 = []
temp2 = []
temp3 = []
temp4 = []
temp5 = []
temp6 = []
temp7 = []
for i in range(len(imu)):
    temp1.append(float(imu[i].header.stamp.secs + imu[i].header.stamp.nsecs * 1e-9))
    temp2.append(float(imu[i].angular_velocity.x))
    temp3.append(float(imu[i].angular_velocity.y))
    temp4.append(float(imu[i].angular_velocity.z))
    temp5.append(float(imu[i].linear_acceleration.x))
    temp6.append(float(imu[i].linear_acceleration.y))
    temp7.append(float(imu[i].linear_acceleration.z))
output_imu = {'timestamp': temp1, 
              'angular_velocity_x': temp2, 'angular_velocity_y': temp3, 'angular_velocity_z': temp4, 
              'linear_acceleration_x': temp5, 'linear_acceleration_y': temp6, 'linear_acceleration_z': temp7}

### gps output

In [40]:
temp1 = []
temp2 = []
temp3 = []
temp4 = []
temp5 = []
for i in range(len(ins)):
    temp1.append(float(ins[i].header.stamp.secs + ins[i].header.stamp.nsecs * 1e-9))
    temp2.append(float(ins[i].latitude))
    temp3.append(float(ins[i].longitude))
    temp4.append(float(ins[i].height))
    temp5.append(float(4.0))
output_gps = {'timestamp': temp1, 
              'latitude': temp2, 'longitude': temp3, 'altitude': temp4, 'position_covariance_type': temp5}

### save matfile

In [41]:
imu_savepath = "output/imu.mat"
gps_savepath = "output/gps.mat"

sio.savemat(imu_savepath, output_imu)
sio.savemat(gps_savepath, output_gps)

In [43]:
load_imu = sio.loadmat(imu_savepath)

### other

In [12]:
bag = rosbag.Bag('/home/crange/dev/calib_ws/calib_data_52/2023-07-12-10-52-56.bag')
save_bag = rosbag.Bag('datas/new_bag.bag', 'w')

for topic, msg, t in bag.read_messages():
    if topic == '/bno055_imu/raw':
        imu_msg = Imu()
        imu_msg.header.stamp = msg.header.stamp
        imu_msg.header.frame_id = msg.header.frame_id
        imu_msg.orientation.x = msg.orientation.x
        imu_msg.orientation.y = msg.orientation.y
        imu_msg.orientation.z = msg.orientation.z
        imu_msg.orientation.w = msg.orientation.w
        imu_msg.orientation_covariance = msg.orientation_covariance
        imu_msg.angular_velocity.x = msg.angular_velocity.x
        imu_msg.angular_velocity.y = msg.angular_velocity.y
        imu_msg.angular_velocity.z = msg.angular_velocity.z
        imu_msg.angular_velocity_covariance = msg.angular_velocity_covariance
        imu_msg.linear_acceleration.x = msg.linear_acceleration.y
        imu_msg.linear_acceleration.y = msg.linear_acceleration.x
        imu_msg.linear_acceleration.z = msg.linear_acceleration.z
        imu_msg.linear_acceleration_covariance = msg.linear_acceleration_covariance
        save_bag.write('/imu/data', imu_msg)
    if topic == '/gps_ori':
        fix_msg = NavSatFix()
        fix_msg = NavSatFix()
        fix_msg.header.stamp = msg.header.stamp
        fix_msg.header.frame_id = msg.header.frame_id
        fix_msg.status.status = msg.status.status
        fix_msg.status.service = msg.status.service
        fix_msg.latitude = msg.latitude
        fix_msg.longitude = msg.longitude
        fix_msg.altitude = msg.altitude
        fix_msg.position_covariance = msg.position_covariance
        fix_msg.position_covariance_type = msg.position_covariance_type
        save_bag.write('/fix', fix_msg)

bag.close()
save_bag.close()